# Teleoperation

In this example we'll control the Jetbot remotely with a gamepad controller connected to our web browser machine.

In [1]:
#setup
import ipywidgets.widgets as widgets
import traitlets
import uuid
import subprocess

from jetbot import Robot
from jetbot import Camera
from jetbot import bgr8_to_jpeg
from jetbot import Heartbeat

#monitor robot connections
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        for link in links:
            link.unlink()
        robot.stop()

#setup camera snapshots
subprocess.call(['mkdir', '-p', 'snapshots'])
snapshot_image = widgets.Image(format='jpeg', width=300, height=300)
def save_snapshot(change):
    # save snapshot when button is pressed down
    if change['new']:
        file_path = 'snapshots/' + str(uuid.uuid1()) + '.jpg'
        # write snapshot to file (we use image value instead of camera because it's already in JPEG format)
        with open(file_path, 'wb') as f:
            f.write(image.value)
        # display snapshot that was saved
        snapshot_image.value = image.value

#start controller
controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

Controller()

In [10]:
#connect robot
robot = Robot()
camera = Camera.instance(fps=10)
image = widgets.Image(format='jpeg', width=300, height=300)
#display(image)

fwd_ax = 1
turn_ax = 2
fwd_tq = 0.3
turn_tq = 0.3
imbal = 1.05

#clear existing links
try:
    while links: links.pop().unlink()
except:
    pass
        
#build links
links = [
    #left stick fwd back right stick turn
    traitlets.dlink((controller.axes[fwd_ax], 'value'), (robot.left_motor, 'value'), transform=lambda x: -fwd_tq*imbal*x),
    traitlets.dlink((controller.axes[fwd_ax], 'value'), (robot.right_motor, 'value'), transform=lambda x: -fwd_tq*x),
    traitlets.dlink((controller.axes[turn_ax], 'value'), (robot.left_motor, 'value'), transform=lambda x: turn_tq*x),
    traitlets.dlink((controller.axes[turn_ax], 'value'), (robot.right_motor, 'value'), transform=lambda x: -turn_tq*x),
    #camera
    traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
]

for b in range(4): controller.buttons[b].observe(lambda data: robot.stop(), names='value') #xyab stop
for c in range(4,17): controller.buttons[c].observe(save_snapshot, names='value') #all other buttons take pictures
    
heartbeat = Heartbeat(period=0.5)
heartbeat.observe(handle_heartbeat_status, names='status') # attach the callback function to heartbeat status
    
display(widgets.HBox([image, snapshot_image]))
display(controller)

Controller(axes=(Axis(value=0.0), Axis(value=0.0), Axis(value=0.0), Axis(value=0.0)), buttons=(Button(value=0.…

In [11]:
while links: links.pop().unlink()
robot.stop()